以下の前処理は  
https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue  
を参照

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import datetime as datetime
from datetime import timedelta, date
import seaborn as sns
import matplotlib.cm as CM
%matplotlib inline

pandas dataframeにjson形式のデータが入っているため, 展開

In [ ]:
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
sub_sample = pd.read_csv("../input/sample_submission.csv")

In [ ]:
sub_sample.head()

In [ ]:
%%time
train_df = load_df()
test_df = load_df("../input/test.csv")

In [ ]:
train_df.head()

すべての行に同じ値が入っているfeatureを抽出

In [ ]:
const_cols = [c for c in train_df.columns if train_df[c].nunique(dropna=False)==1 ]
const_cols

以下のfeatureを特徴量から削除 (セッションIDはデータに対してユニークなので特徴量としては利用できない)  
campaignCodeはtrainにしかなかったため削除  
処理の都合でtrainとtestで異なる値を持つカテゴリカル変数は削除

In [ ]:
cat_cols_all = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']

In [ ]:
add_drop = []
cat_cols = []
for i in range(len(cat_cols_all)):
    if len(set(train_df[cat_cols_all[i]].unique()).difference(set(test_df[cat_cols_all[i]].unique()))) != 0:
        print(cat_cols_all[i], len(set(train_df[cat_cols_all[i]].unique()).difference(set(test_df[cat_cols_all[i]].unique()))), len(train_df[cat_cols_all[i]].unique()))
        add_drop.append(cat_cols_all[i])
    else:
        cat_cols.append(cat_cols_all[i])

In [ ]:
cols_to_drop = const_cols + ['sessionId'] + add_drop

train_df = train_df.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)
test_df = test_df.drop(cols_to_drop, axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0, inplace=True)
#train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float
"""
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
"""
for col in cat_cols:
    print(col)
    train_df[col].fillna(0, inplace=True)
    test_df[col].fillna(0, inplace=True)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col].fillna(0, inplace=True)
    test_df[col].fillna(0, inplace=True)
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)

In [ ]:
index_train = train_df['date'] <= 20170531 #datetime.date(2017,5,31)
index_validation = train_df['date'] > 20170531 #datetime.date(2017,5,31)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
one_hot = OneHotEncoder(sparse = True)
one_hot = one_hot.fit(train_df[cat_cols])
encoded_tr = one_hot.transform(train_df[cat_cols])
encoded_te = one_hot.transform(test_df[cat_cols])

In [ ]:
train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype(float)

In [ ]:
data_tr = np.hstack([train_df[num_cols].values, encoded_tr.toarray()])
data_te = np.hstack([test_df[num_cols].values, encoded_te.toarray()])
y_tr = np.log1p(train_df["totals.transactionRevenue"].values)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
params = {
    "n_estimators" : 500,
    "max_depth": 10,
    "min_samples_split": 10,
    "min_samples_leaf": 10,
    "n_jobs": -1,
    #"random_state ": 2018
}
model = RandomForestRegressor(**params, random_state = 2018, verbose=1)
model = model.fit(data_tr[index_train], y_tr[index_train])


In [ ]:
pred_val = model.predict(data_tr[index_validation])

In [ ]:
from sklearn import metrics
pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":train_id[index_validation]})
val_pred_df["transactionRevenue"] = train_df["totals.transactionRevenue"].values[index_validation]
val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))

In [ ]:
pred_test = model.predict(data_te)

In [ ]:
sub_df = pd.DataFrame({"fullVisitorId":test_id})
pred_test[pred_test<0] = 0
sub_df["PredictedLogRevenue"] = np.expm1(pred_test)
sub_df = sub_df.groupby("fullVisitorId")["PredictedLogRevenue"].sum().reset_index()
sub_df.columns = ["fullVisitorId", "PredictedLogRevenue"]
sub_df["PredictedLogRevenue"] = np.log1p(sub_df["PredictedLogRevenue"])
sub_df.to_csv("baseline_randomforest.csv", index=False)